# Introdução

Busca-se tabular as informações sobre a Divisão Administrativa do Corpo de Bombeiros, visando saber qual contatar, qual a área de abrangência de cada um dos "Grupamentos de Bombeiros" existentes.

No [*site*](http://www.corpodebombeiros.sp.gov.br/) do Corpo de Bombeiros tem mapas em *.jpg* que apresentam a divisão administrativa, indicando qual "Grupamento de Bombeiros" atua em cada um dos 645 municípios do Estado de São Paulo.

Portanto, a idéia é fazer o *download* das imagens, georreferencia-las. Uma vez feito isso, manualmente, definiu-se na tabela de atributos de um *shapefile* qual o ID do "Grupamento de Bombeiros". Uma vez feito isso, torna-se possível "linkar" as informações.


[sss](https://github.com/michelmetran/sp_bombeiro)

In [ ]:
import os
import re
import time
import random
import requests
import numpy as np
import pandas as pd
import geopandas as gpd

from bs4 import BeautifulSoup
from osgeo import gdal, osr
from tqdm.notebook import trange, tqdm

<br>

## *Download* de imagens

Inicialmente definiu-se uma função genérica, para fazer o *download* de arquivos, renomeando-os.

In [ ]:
def download_urls(urls, path, GetFilenameFromURL=True):
    """
    Function to download list of files with a progress bar.
    :param urls: Lista ou tuple, a depender da função #
    :param path: Local onde os arquivos serão inseridos
    :param GetFilenameFromURL: Se VERDADEIRO, a função irá nomear o arquivo conforme o link das lista das URLs.
    Se FALSO, a função irá nomear o arquivo conforme string definida no tuple das URLs
    """
    
    # Reset Interactions
    i = 0
    n_urls = len(urls)

    for n_url in trange(n_urls, desc='Total'):
        # Download path and file name
        if GetFilenameFromURL:
            url = urls[i]
            filename = urls[i].rsplit('/', 1)[1]
        else:
            url = urls[i][0]
            filename = urls[i][1]

        # File size
        r = requests.get(url, stream=True)
        chunk_size = 1024 * 1024
        total_size = int(r.headers['content-length'])

        # Download the file from 'url' and save it locally under 'filename'
        with open(os.path.join(path, filename), 'wb') as f:
            for data in tqdm(
                iterable = r.iter_content(chunk_size=chunk_size),
                total = int(total_size/chunk_size),
                unit = 'MB',
                desc = '{}/{}'.format(str(i+1), str(n_urls))):
                f.write(data)

        # Interactions
        i = i+1

        # Definir um intervalo de tempo
        time.sleep(random.randint(1, 3))

<br>

Usando a biblioteca *BeatifulSoup* foi possível "ler" o código-fonte da página e buscar os *links* para as imagens em *.jpg*. São esses os links que serão utilizados para fazer o *download* dos arquivos na pasta desejada.

In [ ]:
url = 'http://www.corpodebombeiros.sp.gov.br/portalcb/_institucional/'
r = requests.get(url)
soup = BeautifulSoup(r.content, 'html.parser')

In [ ]:
url_partial = '../_institucional/mapas/'

list_urls = []
for i in soup.find_all('a', href=re.compile(url_partial)):
    url_img = os.path.join(
        url,
        os.path.basename(os.path.normpath(url_partial)),
        os.path.basename(os.path.normpath(i['href'])))    
    list_urls.append(url_img)

<br>

Os nomes dos quatro arquivos não agradaram e portanto criei uma lista com os quatro nomes que gostaria de usar. Feito isso, foram definidas *tuplas*, sendo esse o formato que a função que fazer o download e renomeia requer.

In [ ]:
# Set filenames
filenames = ['gb_capital.jpg', 'gb_rmsp.jpg', 'gb_interior.jpg', 'gb_mar.jpg']
dict_urls = {list_urls[i]: filenames[i] for i in range(len(list_urls))}

# Convert Dictionary into Tuples
tuples_urls = dict_urls.items()
tuples_urls = list(tuples_urls)
tuples_urls

In [ ]:
path_bombeiro = os.path.join('data', 'rasters')
download_urls(tuples_urls, path_bombeiro, GetFilenameFromURL=False)

<br>

## Georreference Images

Como as imagens em *.jpg* não tem qualquer refência espacial, foi necessário gerreferencia-las.
Para isso foi utilizada a biblioteca *gdal*.

Para definir os parâmetros necessários para o georreferenciamento (*Ground Points Control*), foi utilizado o QGIS. Alguns problemas de visualização dos mapas no QGIS foram encontrados, por exceder o número de pixels possívels de serem apresentados. Segue o erro:

```bash
#case_divadmin/data/institutions/sp_bombeiro/rasters/gb_capital.jpg, band 3: IReadBlock failed at X offset 0, Y offset 0: Reading this image would require libjpeg to allocate at least 114696000 bytes. This is disabled since above the 104857600 threshold. You may override this restriction by defining the GDAL_ALLOW_LARGE_LIBJPEG_MEM_ALLOC environment variable, or recompile GDAL by defining the GDAL_LIBJPEG_LARGEST_MEM_ALLOC macro to a value greater than 104857600,/home/michel/Documents/Geodata/SourceCode/case_divadmin/d
```

É possível corrigir o problema inserindo o seguinte código no arquivo *.bashrc*. Sendo necessário reiniciar.
```
sudo gedit ~/.bashrc
export GDAL_ALLOW_LARGE_LIBJPEG_MEM_ALLOC=YES
```

Outra alternativa é inserir o código abaixo no "Python Console" na sessão do QGIS.

```python
import gdal
gdal.SetConfigOption('GDAL_ALLOW_LARGE_LIBJPEG_MEM_ALLOC', 'YES')
```



<div class="alert alert-warning">
<b>OBSERVAÇÃO</b><br/>
    Esse <i>post</i> tem a finalidade de mostrar os comandos básicos e me deixar com uma "cola" rápida para meu uso cotidiano.<br/>
    Todas os códigos são exemplificativos e podem/devem ser alterados, indicando o nome dos arquivos e diretórios corretamente.
    Outra alternativa é inserir o código abaixo no "Python Console" na sessão do QGIS.

```python
import gdal
gdal.SetConfigOption('GDAL_ALLOW_LARGE_LIBJPEG_MEM_ALLOC', 'YES')
```


</div>

<div class="alert alert-info">
<b>INFORMAÇÃO</b><br/>
    <ol>
    <li>É possível acessar esse <i>post</i> em formato <a href="https://rawcdn.githack.com/michelmetran/package_pandas/master/docs/pandas.html" target="_blank"><i><b>html</b></i></a>, que possibilita ter uma visualização rápida do código;</li>
    <li>Diretamente por meio do <a href="https://github.com/michelmetran/package_pandas" target="_blank"><b>repositório</b></a>, onde está disponível este arquivo <i><b>.ipynb</b></i>, que permite fazer edições no código;</li>
    <li>Ou ainda, de maneira interativa, usando o <a href="https://mybinder.org/v2/gh/michelmetran/package_pandas/master" target="_blank"><i><b>MyBinder</b></i></a>, que possibilita rodar e editar o código sem a necessidade de instalar nada.</li>
    </ol>
</div>

### Função

In [ ]:
def georeferencing(src_filename, dst_filename, gcp_list):
    # 
    src_ds = gdal.Open(src_filename, gdal.GA_ReadOnly)
    driver = gdal.GetDriverByName('JPEG')

    # Open destination dataset
    dst_ds = driver.CreateCopy(dst_filename, src_ds, 0)

    # Get raster projection
    srs = osr.SpatialReference()
    srs.ImportFromEPSG(4674)
    dst_wkt = srs.ExportToWkt()

    # Apply the GCPs to the open output file:
    dst_ds.SetGCPs(gcp_list, dst_wkt)

    # Warp Image
    dst_ds = gdal.Warp(dst_filename, dst_ds, dstSRS=dst_wkt)

    # Close files
    src_ds = None
    dst_ds = None
    
    print('Arquivo "{}" georreferenciado.'.format(src_filename))

### Interior

In [ ]:
# Interior
gcp_interior = []
gcp_interior.append(gdal.GCP(-53.088, -22.658, 0, 94,   1085))
gcp_interior.append(gdal.GCP(-44.161, -22.678, 0, 2922, 1092))
gcp_interior.append(gdal.GCP(-48.897, -20.442, 0, 1422, 384))
gcp_interior.append(gdal.GCP(-48.529, -25.101, 0, 1538, 1860))

georeferencing(
    os.path.join(path_bombeiro, 'gb_interior.jpg'),
    os.path.join(path_bombeiro, 'geo_gb_interior.jpg'),
    gcp_interior,
)

### RMSP

In [ ]:
# RMSP
gcp_rmsp = []
gcp_rmsp.append(gdal.GCP(-47.19992445343228127,-24.00057005235778007,0,46.22207446808516806,1710.09109042553200197))
gcp_rmsp.append(gdal.GCP(-45.69482925159626063,-23.63079105379386391,0,2988.91260804521380123,990.57858211436268903))
gcp_rmsp.append(gdal.GCP(-46.53544614046614214,-23.36327027147718383,0,1344.75577626329209124,468.2524517952102201))
gcp_rmsp.append(gdal.GCP(-46.77483081914691354,-24.00324657360952685,0,878.35251828456978274,1718.67324634307919951))

georeferencing(
    os.path.join(path_bombeiro, 'gb_rmsp.jpg'),
    os.path.join(path_bombeiro, 'geo_gb_rmsp.jpg'),
    gcp_rmsp,
)

### Capital

In [ ]:
# Capital
gcp_capital = []
gcp_capital.append(gdal.GCP(-46.54915525056139103,-23.35609585395840426,0,1767.96235679214441916,173.52373158756142857))
gcp_capital.append(gdal.GCP(-46.77240003062176044,-24.0062705463104642,0,515.63666121113021745,3819.83960720131017297))
gcp_capital.append(gdal.GCP(-46.57900155611972082,-23.59953641000311464,0,1601.97054009820453757,1538.68330605564642614))
gcp_capital.append(gdal.GCP(-46.79950529616165511,-23.47128676778620004,0,363.83960720131642574,819.09574468085168064))

georeferencing(
    os.path.join(path_bombeiro, 'gb_capital.jpg'),
    os.path.join(path_bombeiro, 'geo_gb_capital.jpg'),
    gcp_capital,
)

### Mar

In [ ]:
# Mar
gcp_mar = []
gcp_mar.append(gdal.GCP(-48.0989060186612889,-25.31117216047202234,0,240.17103109656343918,1854.25378477905019281))
gcp_mar.append(gdal.GCP(-44.72370514232069638,-23.3673844646437594,0,2484.78403743863009367,561.29569864975394466))
gcp_mar.append(gdal.GCP(-46.39932135348571762,-24.03377814564129267,0,1370.76659676759936701,1003.88551043371489868))

georeferencing(
    os.path.join(path_bombeiro, 'gb_mar.jpg'),
    os.path.join(path_bombeiro, 'geo_gb_mar.jpg'),
    gcp_mar,
)

<br>

## Dados Espaciais

Usando o código *get_spatial_data.ipynb* foi possível baixar a malha municipal do estado de São Paulo do [*site* do IBGE](https://geoftp.ibge.gov.br/organizacao_do_territorio/malhas_territoriais/malhas_municipais/municipio_2020/UFs/SP/SP_Municipios_2020.zip). Nos códigos abaixo o arquivo *shapefile* é lido, são feitos pequenos ajustes para ter um material mais "limpo" para definir, manualmente, quais os "Grupamentos de Bombeiro" que atual em cada um dos municípios.

In [ ]:
# Lê o arquivo shapefile
file = 'sp_ibge_250k'
shp = os.path.join('data', 'shps', '{}.shp'.format(file))
gdf = gpd.read_file(shp)

# Deleta colunas
gdf.drop(['SIGLA_UF', 'AREA_KM2'], axis=1, inplace=True)

# Renomeia colunas
gdf.rename(columns={'CD_MUN':'id_municipio'}, inplace=True)
gdf.rename(columns={'NM_MUN':'municipio_nome'}, inplace=True)

# Reordena colunas
gdf['ib_gb'] = 1
gdf = gdf[['id_municipio', 'municipio_nome', 'ib_gb', 'geometry']]

# ATENÇÃO: SÓ EXECUTAR SE FOR REFAZER A CLASSIFICAÇÃO MANUALMENTE NO QGIS!
#gdf.to_file(os.path.join('data', 'shps', 'div_admin' + '.shp'), encoding='utf-8')

# Results
gdf.head()

<br>

## Dados Tabulares

Por meio do [*site* oficial](http://www.corpodebombeiros.sp.gov.br/portalcb/_institucional/) foi possível obter as informações sobre os "Comandos", ou seja, a estrutura hierárquica da instituição.

Em *sites* não oficiais, tais como o [*site* dos bombeiros socorristas](http://www.bombeirossocorristas.com.br/dicas?id=99/departamentos_do_corpo_de_bombeiros_da_policia_militar_do_estado_de_sao_paulo_), foi possível obter o endereço dos "Grupamentos de Bombeiros".

In [ ]:
# Lê o arquivo csv com todos os atributos de cada um dos Grupamentos de Bombeiros
df = pd.read_csv(
    os.path.join('data', 'tabs', 'tab_bombeiro.csv'),    
).fillna(value=np.NaN)

# Define zero
df['numero'] = df['numero'].fillna(0)

# Convert para Inteiro
df['numero'] = df['numero'].astype(int)

# Results
df.head()

In [ ]:
# Lê o arquivo shapefile com os códigos dos municípios e ID do Grupamento de Bombeiros
gdf = gpd.read_file(
    os.path.join('data', 'shps', 'div_admin.shp'),
)
# Select Columns
gdf = gdf[['id_mun', 'id_gb', 'id_gbmar']]

gdf['id_mun'] = gdf['id_mun'].astype(int)

# Results
gdf.head()

In [ ]:
# Lê o arquivo csv com o nome dos municípios
df_mun = pd.read_csv(
    'https://raw.githubusercontent.com/michelmetran/sp/main/data/tabs/tab_municipios.csv',
)
df_mun = df_mun[['id_mun', 'municipio_nome']]
df_mun

In [ ]:
# Junta as informações e salta um arquivo final
df_bomb = pd.merge(
    gdf,
    df,
    how='left',
    left_on='id_gb',
    right_on='id_gb'
)
df_bomb

In [ ]:
pd.merge(
    df_mun,
    df_bomb,
    how='left',
    left_on='id_mun',
    right_on='id_mun'
)

In [ ]:

# Save
df_bomb.to_csv(
    os.path.join('data', 'tabs', 'tab_municipios_bombeiros.csv'),    
    index=False
)

# Results
df_bomb.head()

___

# Encerramento

## *Requirements*

In [ ]:
# Input
inp = 'get_infos.ipynb'

# Output
out = os.path.join(inp.split('.')[0])
!jupyter-nbconvert $inp --to python --output $out

print('-'*80)
!pipreqs '.' --force --debug

print('-'*80)
os.remove('{}.py'.format(out))
print('Arquivo {}.py deletado com sucesso!'.format(out))

## *GitHub*

In [ ]:
!nbstripout --install --attributes .gitattributes
!git status
!git add .
!git commit -m 'Arrumações Gerais'
!git push origin main